# Ball tracking with shape

### Sources
- https://www.youtube.com/watch?v=RaCwLrKuS1w
- https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

In [1]:
# Import OpenCV, a software used for Computer Vision
import cv2 as cv

import numpy as np
import pandas as pd
import os

## Parameters that can be changed

In [2]:
ballType = 1  # 1 = tennis ball, 2 = bouncing ball

videoName = "Videos/ball1_3.mp4"  # If the argument is set to 0, read the video from the
                                                             # webcam. Otherwise, the argument should be the path to
                                                             # the video to be read.
saveName = "video.avi" # name of the video file to store the video
dataName = "data.csv" # name of the data file to store the extracted coordinates

fps = 24  # frames per second

cropping = True  # If set to True, the video will be cropped. Otherwise, the video stays in its original state.

# default frame dimensions (depend on the camera we use)
width = 1920
height = 1080

# cropping settings
if cropping:
    lower = 0
    upper = 1200
    left = 700
    right = 1100
    
    # frame dimensions
    width = right - left
    height = upper - lower
    
# ball selection
if ballType == 1:  # tennis ball
    minrad=20
    maxrad=25
elif ballType == 2:  # bouncing ball
    minrad=10
    maxrad=15
    

In [3]:
## DATA STORAGE:
if videoName[0:7]=="Videos/":
    storageName=videoName[7:-4] #remove the folder name and format .mp4
    print(storageName)

path=os.getcwd()+"/TrackingResults/Shape/"+str(storageName)

isExist = os.path.exists(path)     # Check whether the specified path exists or not
if not isExist:                    # Create a new directory because it does not exist
    os.makedirs(path)
    print("The new directory is created!")
f=open("TrackingResults/Shape/"+str(storageName)+"/parameters.txt",'w+')
f.write("cropping "+str(cropping)+"\n"+"balltype "+str(ballType)+"\n"+"videoname "+str(videoName)+"\n"+"fps "+str(fps)+"\n")
f.close()
saveName = path+"/"+saveName
dataName = path+"/"+dataName

ball1_3


## Display video with tracking circle & Save video and position data

In [4]:
# create a VideoCapture object to display the video
cap = cv.VideoCapture(videoName)

print("frame size = " + str(cap.get(cv.CAP_PROP_FRAME_WIDTH)) + " x " + str(cap.get(cv.CAP_PROP_FRAME_HEIGHT)))

# define the codec and create a VideoWriter object used to record the video
fourcc = cv.VideoWriter_fourcc(*'MJPG')
out = cv.VideoWriter(saveName, fourcc, fps/2, (width, height)) # specify the output video name (1), the fps (3) and the frame
                                                               # size (4)

# create arrays to store data
x_vec = []
y_vec = []

prevCircle = None
dist = lambda x1,y1,x2,y2: (x1-x2)**2+(y1-y2)**2 # square distance between points (x1,y1) and (x2,y2)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    # If frame is read correctly ret is True.
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # crop the video to reduce the area for detection
    if cropping:
        frame = frame[lower:upper, left:right]
    
    # change the frame from colored to grayscale
    grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # blur the frame to reduce noise and outliers
    blurFrame = cv.GaussianBlur(grayFrame, (17,17), 1) # argument (3) allows to blur more the frame by increasing its value
    
    # use Hough Circle Transform to detect circles in an image (only in grayscale images)
    circles = cv.HoughCircles(blurFrame, cv.HOUGH_GRADIENT, 1.2, 100, param1=90, param2=15, minRadius=minrad, maxRadius=maxrad)
    # increasing dp (3) allows for closer circles to be merged
    # (4) = minDist between the centers of the detected circles
    # (5) = sensitivity of circles detection => higher to detect less circles
    # use (6) to set the min edges to say it is a circle => higher to detect less
    
    if circles is not None:
        
        circles = np.uint16(np.around(circles))
        chosen = None
        
        for i in circles[0,:]:
            
            if chosen is None:
                chosen = i
            
            if prevCircle is not None:
                if dist(chosen[0], chosen[1], prevCircle[0], prevCircle[1]) <= dist(i[0], i[1], prevCircle[0], prevCircle[1]):
                    chosen = i
        
        cv.circle(frame, (chosen[0], chosen[1]), 1, (0, 100, 100), 3) # draw circle center
        cv.circle(frame, (chosen[0], chosen[1]), chosen[2], (255, 0, 255), 3) # draw circle (chosen[2] = radius)
        prevCircle = chosen
    
        # store the ball center position in the arrays
        x_vec.append(chosen[0])
        y_vec.append(chosen[1])
    
    # change the frame size which will be recorded
    frame = cv.resize(frame, (width, height), fx=0, fy=0, interpolation = cv.INTER_CUBIC)
    # record frame
    out.write(frame)
    
    # change the frame size which will be displayed
    frame = cv.resize(frame, (300,750))
    # display the resulting frame
    cv.imshow("Frame", frame)
    
    # press Q to stop the video
    if cv.waitKey(fps*2) & 0xFF == ord('q'): # change argument of cv.waitKey() for slow motion
        break
    
# save the storing arrays in a csv file
df = pd.DataFrame({"x": x_vec, "y": y_vec})
df.to_csv(dataName, mode="w", index=False, sep=',')

# Release everything when job is finished
cap.release() # stop displaying video
out.release() # stop recording

cv.destroyAllWindows()

frame size = 1920.0 x 1088.0


C:\Users\kiwix\AppData\Local\Temp/ipykernel_41368/1256205331.py:16: RuntimeWarning: overflow encountered in ushort_scalars
  dist = lambda x1,y1,x2,y2: (x1-x2)**2+(y1-y2)**2 # square distance between points (x1,y1) and (x2,y2)


Can't receive frame (stream end?). Exiting ...
